### Week 6, Day 2

We're about to create and use our own MCP Server and MCP Client!

It's pretty simple, but it's not super-simple. The excitment around MCP is about how easy it is to share and use other MCP Servers - making our own does involve a bit of work.

Let's review some python code made mostly by a hard-working Engineering Team:

accounts.py

In [12]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio, MCPServerStdioParams
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [13]:
from accounts import Account

In [14]:
account = Account.get("Quoc")
account

Account(name='quoc', balance=9717.436, strategy='', holdings={'AMZN': 6}, transactions=[3 shares of AMZN at 91.182 each., 3 shares of AMZN at 3.0060000000000002 each.], portfolio_value_time_series=[('2025-07-14 17:30:20', 9765.454), ('2025-07-14 17:30:23', 9816.454), ('2025-07-14 17:37:43', 9915.436), ('2025-07-14 17:37:45', 9891.436)])

In [15]:
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "quoc", "balance": 9458.92, "strategy": "", "holdings": {"AMZN": 9}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 91.182, "timestamp": "2025-07-14 17:30:20", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 3.0060000000000002, "timestamp": "2025-07-14 17:37:43", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 86.172, "timestamp": "2025-07-14 17:38:14", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-07-14 17:30:20", 9765.454], ["2025-07-14 17:30:23", 9816.454], ["2025-07-14 17:37:43", 9915.436], ["2025-07-14 17:37:45", 9891.436], ["2025-07-14 17:38:14", 9962.92]], "total_portfolio_value": 9962.92, "total_profit_loss": -37.07999999999993}'

In [16]:
account.report()

'{"name": "quoc", "balance": 9458.92, "strategy": "", "holdings": {"AMZN": 9}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 91.182, "timestamp": "2025-07-14 17:30:20", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 3.0060000000000002, "timestamp": "2025-07-14 17:37:43", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 86.172, "timestamp": "2025-07-14 17:38:14", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-07-14 17:30:20", 9765.454], ["2025-07-14 17:30:23", 9816.454], ["2025-07-14 17:37:43", 9915.436], ["2025-07-14 17:37:45", 9891.436], ["2025-07-14 17:38:14", 9962.92], ["2025-07-14 17:38:16", 9575.92]], "total_portfolio_value": 9575.92, "total_profit_loss": -424.0799999999999}'

In [17]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 91.182,
  'timestamp': '2025-07-14 17:30:20',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 3.0060000000000002,
  'timestamp': '2025-07-14 17:37:43',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 86.172,
  'timestamp': '2025-07-14 17:38:14',
  'rationale': 'Because this bookstore website looks promising'}]

### Now we write an MCP server and use it directly!

In [21]:
# Now let's use our accounts server as an MCP server

params = MCPServerStdioParams(command="uv", args=["run", "accounts_server.py"])
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()


In [22]:
mcp_tools

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None),
 Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None),
 Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', '

In [23]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Quoc and my account is under the name Quoc. What's my balance and my holdings?"
model = "gpt-4.1-mini"

In [24]:

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))


Quoc, your current account balance is $9,458.92. You hold 9 shares of AMZN (Amazon) stock. If you need any further information or assistance, feel free to ask!

### Now let's build our own MCP Client

In [25]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None), Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None), Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', 'ty

In [29]:
request = "My name is Quoc and my account is under the name Quoc. What's my balance and holdings?"

with trace("account_mcp_client"):
    agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Quoc, your account balance is $9,458.92. Your current holdings include 9 shares of Amazon (AMZN). If you have any further questions or would like to make transactions, just let me know!

In [30]:
context = await read_accounts_resource("Quoc")
print(context)

{"name": "quoc", "balance": 9458.92, "strategy": "", "holdings": {"AMZN": 9}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 91.182, "timestamp": "2025-07-14 17:30:20", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 3.0060000000000002, "timestamp": "2025-07-14 17:37:43", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 86.172, "timestamp": "2025-07-14 17:38:14", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-07-14 17:30:20", 9765.454], ["2025-07-14 17:30:23", 9816.454], ["2025-07-14 17:37:43", 9915.436], ["2025-07-14 17:37:45", 9891.436], ["2025-07-14 17:38:14", 9962.92], ["2025-07-14 17:38:16", 9575.92], ["2025-07-14 17:52:29", 9557.92]], "total_portfolio_value": 9557.92, "total_profit_loss": -442.0799999999999}


In [31]:
from accounts import Account
Account.get("Quoc").report()

'{"name": "quoc", "balance": 9458.92, "strategy": "", "holdings": {"AMZN": 9}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 91.182, "timestamp": "2025-07-14 17:30:20", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 3.0060000000000002, "timestamp": "2025-07-14 17:37:43", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 86.172, "timestamp": "2025-07-14 17:38:14", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-07-14 17:30:20", 9765.454], ["2025-07-14 17:30:23", 9816.454], ["2025-07-14 17:37:43", 9915.436], ["2025-07-14 17:37:45", 9891.436], ["2025-07-14 17:38:14", 9962.92], ["2025-07-14 17:38:16", 9575.92], ["2025-07-14 17:52:29", 9557.92], ["2025-07-14 17:53:15", 9800.92]], "total_portfolio_value": 9800.92, "total_profit_loss": -199.07999999999993}'

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercises</h2>
            <span style="color:#ff7800;">Make your own MCP Server! Make a simple function to return the current Date, and expose it as a tool so that an Agent can tell you today's date.<br/>Harder optional exercise: then make an MCP Client, and use a native OpenAI call (without the Agents SDK) to use your tool via your client.
            </span>
        </td>
    </tr>
</table>

In [36]:
params = MCPServerStdioParams(command="uv", args=["run", "date_server.py"])
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

In [33]:
mcp_tools

[Tool(name='get_date_today', description="Get today's date in the format YYYY-MM-DD.", inputSchema={'properties': {}, 'title': 'get_date_todayArguments', 'type': 'object'}, annotations=None)]

In [38]:
date_getter_instructions = "You are able to get today's date."
request = "What is today's date?"
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as date_server:
    agent = Agent(name="date_getter", instructions=date_getter_instructions, model=model, mcp_servers=[date_server])
    with trace("date_getter"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Today's date is July 14, 2025.